In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import sys
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#Validation
def dataset_validation(loader, model):
  class_labels = []
  class_preds = []
  with torch.no_grad():
      n_correct = 0
      n_samples = 0
      for images, labels in loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          # max returns (value ,index)
          values, predicted = torch.max(outputs.data, 1)
          n_samples += labels.size(0)
          n_correct += (predicted == labels[:,1]).sum().item()

      acc = 100.0 * n_correct / n_samples
  return acc

#Generate new_filename
def generate_folder_name(Name_NN, directory):
    count = 1
    FolderName = Name_NN

    while os.path.exists(os.path.join(directory, FolderName)):
        # If file exist append a number to the name
        FolderName = f"{Name_NN}_{count}"
        count += 1

    return FolderName

In [ ]:
####################################GUARDADO##############################################
#RUTA GENERAL DE GUARDADO:
#path_NN='/content/drive/MyDrive/Colab Notebooks/ARCHIVO_FINAL_PROYECTO_RL/NN/'
path_TENSORBOARD = '/content/drive/MyDrive/Colab Notebooks/PROYECT_EMOTION/tensorboard_emot'
#Ruta de guardado
Name_NN="CNN"
FolderName = generate_folder_name(Name_NN, path_TENSORBOARD)
print(FolderName)
##########################################################################################


#Tensorboard
writer = SummaryWriter('/content/drive/MyDrive/Colab Notebooks/PROYECT_EMOTION/tensorboard_emot/' + FolderName)

%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Colab\ Notebooks/PROYECT_EMOTION/tensorboard_emot


CNN_9


In [ ]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x = []
        self.y = []

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        sample = self.x[idx], self.y[idx]
        return sample

train_dataset = CustomDataset()
test_dataset = CustomDataset()

#Dataset
path_data = '/content/drive/MyDrive/Colab Notebooks/PROYECT_EMOTION/data_norm/samples.npy'
path_labels = '/content/drive/MyDrive/Colab Notebooks/PROYECT_EMOTION/data_norm/valence_labels_bce.npy'
train_dataset.x = torch.from_numpy(np.load(path_data).astype(np.float32))
train_dataset.y = torch.from_numpy(np.load(path_labels).astype(np.float32) )

train_dataset.x, test_dataset.x, train_dataset.y, test_dataset.y = train_test_split(
    train_dataset.x, train_dataset.y, test_size=0.1, random_state = True, stratify = train_dataset.y)

In [ ]:
print('Train sample shape:', train_dataset.x.size())
print('Test sample shape:', test_dataset.x.size())
print('Train sample type:', train_dataset.x.dtype)
print('Test sample type:', test_dataset.x.dtype)

print('Train labels shape:', train_dataset.y.size())
print('Test labels shape:', test_dataset.y.size())
print('Train labels type:', train_dataset.y.dtype)
print('Test labels type:', test_dataset.y.dtype)


count_1 = 0
count_0 = 0
len_train = train_dataset.__len__()
for i in range(len_train):
  if(train_dataset.y[i, 0] == 1):
    count_0+=1
  else:
    count_1+=1

print('cantidad ceros:', count_0, 'cantidad de 1:', count_1)

count_1 = 0
count_0 = 0
len_train = test_dataset.__len__()
for i in range(len_train):
  if(test_dataset.y[i, 0] == 1):
    count_0+=1
  else:
    count_1+=1

print('cantidad ceros:', count_0, 'cantidad de 1:', count_1)

Train sample shape: torch.Size([24192, 1, 384, 32])
Test sample shape: torch.Size([2688, 1, 384, 32])
Train sample type: torch.float32
Test sample type: torch.float32
Train labels shape: torch.Size([24192, 2])
Test labels shape: torch.Size([2688, 2])
Train labels type: torch.float32
Test labels type: torch.float32
cantidad ceros: 13873 cantidad de 1: 10319
cantidad ceros: 1541 cantidad de 1: 1147


In [ ]:
# Hyper-parameters
num_epochs = 200
batch_size = 128
learning_rate = 0.0001
Momentum = 0.9
Dropout = 0.25

#DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)


In [ ]:
#Creation of the CNN
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Layer 1: Input
        self.conv1 = nn.Conv2d(1, 25, kernel_size=(5, 1))
        self.dropout1 = nn.Dropout2d(p=0.25)
        # Layer 2: conv_2
        self.conv2 = nn.Conv2d(25, 25, kernel_size=(1, 3), stride=(1, 2))
        self.bn1 = nn.BatchNorm2d(25, momentum = 0.9)
        self.pool1 = nn.MaxPool2d((2, 1))
        # Layer 3: conv_3
        self.conv3 = nn.Conv2d(25, 50, kernel_size=(5, 1))
        self.dropout2 = nn.Dropout2d(p=0.25)
        # Layer 4: conv_4
        self.conv4 = nn.Conv2d(50, 50, kernel_size=(1, 3), stride=(1, 2))
        self.bn2 = nn.BatchNorm2d(50, momentum = 0.9)
        self.pool2 = nn.MaxPool2d((2, 1))
        # Layer 5: conv_5
        self.conv5 = nn.Conv2d(50, 100, kernel_size=(5, 1))
        self.dropout3 = nn.Dropout2d(p=0.25)
        # Layer 6: conv_6
        self.conv6 = nn.Conv2d(100, 100, kernel_size=(1, 3), stride=(1, 2))
        self.bn3 = nn.BatchNorm2d(100, momentum = 0.9)
        self.pool3 = nn.MaxPool2d((2, 1))
        # Layer 7: conv_7
        self.conv7 = nn.Conv2d(100, 200, kernel_size=(5, 1))
        self.dropout4 = nn.Dropout2d(p=0.25)
        # Layer 8: conv_8
        self.conv8 = nn.Conv2d(200, 200, kernel_size=(1, 3))
        self.bn4 = nn.BatchNorm2d(200, momentum = 0.9)
        # Flatten layer
        self.flatten = nn.Flatten()
        # Linear layers
        self.fc1 = nn.Linear( 8000, 256)
        self.dropout5 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 2)  # Output for binary classification task

    def forward(self, x):
        # Layer 1: Input
        x = F.leaky_relu(self.conv1(x), negative_slope=0.3) #negative slope equals to alpha
        x = self.dropout1(x)
        # Layer 2: conv_2
        x = F.leaky_relu(self.conv2(x), negative_slope=0.3)
        x = self.bn1(x)
        x = self.pool1(x)
        # Layer 3: conv_3
        x = F.leaky_relu(self.conv3(x), negative_slope=0.3)
        x = self.dropout2(x)
        # Layer 4: conv_4
        x = F.leaky_relu(self.conv4(x), negative_slope=0.3)
        x = self.bn2(x)
        x = self.pool2(x)
        # Layer 5: conv_5
        x = F.leaky_relu(self.conv5(x), negative_slope=0.3)
        x = self.dropout3(x)
        # Layer 6: conv_6
        x = F.leaky_relu(self.conv6(x), negative_slope=0.3)
        x = self.bn3(x)
        x = self.pool3(x)
        # Layer 7: conv_7
        x = F.leaky_relu(self.conv7(x), negative_slope=0.3)
        x = self.dropout4(x)
        # Layer 8: conv_8
        x = F.leaky_relu(self.conv8(x), negative_slope=0.3)
        x = self.bn4(x)
        # Flatten
        x = self.flatten(x)
        # Linear layers
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.dropout5(x)
        x = self.fc2(x)
        #Normalization with softmax layer
        x = F.softmax(x, dim = 1)

        return x
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Run model on the GPU
model = CustomCNN().to(device)

examples = iter(test_loader)
example_data, example_targets = next(examples)



writer.add_graph(model, example_data.to(device))
writer.close()
#sys.exit()


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Model evaluation
loss_evol = []
running_loss = 0
running_correct = 0

#Print/iterations
iter_print = 79

n_total_steps = len(train_loader) #sample number / batch size --> 41948/128
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        #signals.reshape(128, 1, 32, 384)
        signals = signals.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(signals)
        #loss = criterion(m(outputs), labels)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Display
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        running_correct += (predicted == labels[:, 1]).sum().item()

        if (i+1) % iter_print == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

            #Tensorboard
            writer.add_scalar('training loss', running_loss / iter_print, epoch * n_total_steps + i)
            running_accuracy = running_correct / iter_print / predicted.size(0)
            writer.add_scalar('accuracy train dataset', running_accuracy, epoch * n_total_steps + i)
            running_correct = 0
            running_loss = 0.0

    train_acc = dataset_validation(train_loader, model)
    test_acc = dataset_validation(test_loader, model)
    print (f'Epoch [{epoch+1}],  Accuracy on the test dataset: {train_acc}, Accuracy on the test dataset: {test_acc}')



print('Finished Training')
PATH = path_TENSORBOARD + FolderName
torch.save(model.state_dict(), PATH)

Epoch [1/200], Step [79/189], Loss: 0.7181
Epoch [1/200], Step [158/189], Loss: 0.7306
Epoch [1],  Accuracy on the test dataset: 55.410879629629626, Accuracy on the test dataset: 53.236607142857146
Epoch [2/200], Step [79/189], Loss: 0.6612
Epoch [2/200], Step [158/189], Loss: 0.6827
Epoch [2],  Accuracy on the test dataset: 56.73776455026455, Accuracy on the test dataset: 54.055059523809526
Epoch [3/200], Step [79/189], Loss: 0.6937
Epoch [3/200], Step [158/189], Loss: 0.6722
Epoch [3],  Accuracy on the test dataset: 57.30820105820106, Accuracy on the test dataset: 56.54761904761905
Epoch [4/200], Step [79/189], Loss: 0.7198
Epoch [4/200], Step [158/189], Loss: 0.6575
Epoch [4],  Accuracy on the test dataset: 58.42427248677249, Accuracy on the test dataset: 57.961309523809526
Epoch [5/200], Step [79/189], Loss: 0.6850
Epoch [5/200], Step [158/189], Loss: 0.6578
Epoch [5],  Accuracy on the test dataset: 59.664351851851855, Accuracy on the test dataset: 57.03125
Epoch [6/200], Step [79/

KeyboardInterrupt: 

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
class_labels = []
class_preds = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        values, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        class_probs_batch = [F.softmax(output, dim=0) for output in outputs]

        class_preds.append(class_probs_batch)
        class_labels.append(labels)

    # 10000, 10, and 10000, 1
    # stack concatenates tensors along a new dimension
    # cat concatenates tensors in the given dimension
    class_preds = torch.cat([torch.stack(batch) for batch in class_preds])
    class_labels = torch.cat(class_labels)

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test images: {acc} %')

    ############## TENSORBOARD ########################
    classes = range(2)
    for i in classes:
        labels_i = class_labels == i
        preds_i = class_preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        writer.close()
    ###################################################
